In [2]:
from final_project_group_13 import *

# Test dataset loader from paper code

In [15]:
DATASET_PATH = r'C:\Users\Michael Metz\Documents\Education\MEng Software\2023-01 - 2023-04 Winter\ENEL 645\BreakHis\BreaKHis_v1\BreaKHis_v1\histology_slides\breast'

ds : tf.data.Dataset = dataset(DATASET_PATH, train=False)

y_labels = []
for image_batch, label_batch in ds:
    y_labels.append(label_batch)

y_labels = tf.concat(y_labels, axis=0)
y_labels.numpy()

Found 7909 files belonging to 2 classes.


<tf.Tensor: shape=(7909, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [19]:
y_labels_arr = y_labels.numpy()

print(*y_labels_arr, sep='\n')

[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]


In [13]:
import math
import random
import shutil
from glob import glob
from pathlib import Path

import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
import pickle
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0


# 1. Configuration

In [14]:
"""
The path where the two class folders (benign, malignant) of images reside.

raw strings are used in-case you use a Windows path with `\`.

If you want any other paths in this script to be cross platform, you *must* use
the forward slash `/` to make the paths work on Linux or Mac. But if you are
just using a path on only a Windows machine (like this DATASET_PATH) you can
use `\`.

Also note you can't end a raw string with a `\` (and don't need to in this case
as we just need the path up to the folder)

"""
DATASET_PATH = Path(
    r'C:\Users\Michael Metz\Documents\Education\MEng Software\2023-01 - 2023-04 Winter\ENEL 645\BreakHis\BreaKHis_v1\BreaKHis_v1\histology_slides\breast'
)
# DATASET_PATH = Path(r'/mnt/c/Users/Michael Metz/Documents/Education/MEng Software/2023-01 - 2023-04 Winter/ENEL 645/BreakHis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast')

"""
Set the train, validation, test split. These must add up to approximately 1.0.
"""
TRAIN_SPLIT = 0.6
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 1.0 - VALIDATION_SPLIT - TRAIN_SPLIT

"""
Used for configuring a consistent RANDOM_SEED where we need randomness with
reproducable results, like when shuffling the order of images.
"""
RANDOM_SEED = 154

"""
Enable memory growth got the first GPU detected instead of assigning all
of the device memory during runtime initialization
"""
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


# 2. Stratify Split the data into development and test datasets

In [53]:
CLASS_LIST = ['benign', 'malignant']
SPLIT_DATASET_PATH = DATASET_PATH / 'split_data'


def stratified_train_val_test_split_into_folders(
        dataset_path=DATASET_PATH,
        *,
        class_list=CLASS_LIST,
        split_data_path=DATASET_PATH / 'split_data',
        move=False,
        train_split=TRAIN_SPLIT,
        validation_split=VALIDATION_SPLIT,
        test_split=TEST_SPLIT,
        random_seed=RANDOM_SEED
):
    """
    Loops through the `class_list` and splits the data set into train, test,
    and validation datasets. The images will be in `split_data_path`/`

    Args:
        dataset_path (Path, optional): The folder that contains the class folders with pngs in the class folders or any folder below. Defaults to DATASET_PATH.
        class_list (list, optional): List of expected class subfolders. Defaults to CLASS_LIST.
        split_data_path (Path, optional): Where to output the split data. Defaults to DATASET_PATH/'split_data'.
        move (bool, optional): Move files from `dataset_path` if True, else copy the files. Defaults to False.
        train_split (float, optional): Amount to split into training. Defaults to TRAIN_SPLIT.
        validation_split (float, optional): Amount to split into validation. Defaults to VALIDATION_SPLIT.
        test_split (float, optional): Amount to split into test. Defaults to TEST_SPLIT.
        random_seed (int, optional): random seed to use for shuffling. Defaults to RANDOM_SEED.

    Raises:
        ValueError: All splits must add up to approximately 1.0, if they don't this is raised.

    Returns:
        list(str): A list of strings, one each for train, validation, test path.
    """

    TRAIN_FOLDER_NAME = 'training'
    VALIDATION_FOLDER_NAME = 'validation'
    TEST_FOLDER_NAME = 'test'

    split_total = train_split + validation_split + test_split
    EXPECTED_SPLIT_TOTAL = 1.0
    if not math.isclose(split_total, EXPECTED_SPLIT_TOTAL):
        raise ValueError(
            'train_split + validation_split + test_split ({}) is not approximately = {}'.format(split_total, EXPECTED_SPLIT_TOTAL))

    copy_move_str = 'Copying'
    if move:
        copy_move_str = 'Moving'

    development_split = train_split + validation_split

    destination_paths = []

    allow_move_or_copy = True

    if split_data_path.exists():
        print(
            f"Not {copy_move_str.lower()} files as {split_data_path} already esists")
        allow_move_or_copy = False

    split_data_path.mkdir(parents=True, exist_ok=True)

    for class_index, class_ in enumerate(class_list):
        class_images = glob(
            str(dataset_path / class_ / '**/*.png'), recursive=True)

        # Shuffles the list in place.
        random.Random(random_seed).shuffle(class_images)

        development_length = int(development_split * len(class_images))

        print(f'Development {class_} set length: {development_length}')
        print(
            f'Test {class_} set length: {len(class_images) - development_length}')

        development_class_image_paths = class_images[:development_length]
        test_class_image_paths = class_images[development_length:]

        print(
            f'Development {class_} image count: {len(development_class_image_paths)}')
        print(f'Test {class_} image count: {len(test_class_image_paths)}')

        '''
        / does float division in python3 and we expect these numbers to be float
        anyways.

        TRAIN_SPLIT is relative to DEVELOPMENT_SPLIT images because we are working
        with an images subset, and the numbers are absolute to the total dataset.
        '''
        training_length = int(train_split / development_split *
                              len(development_class_image_paths))

        print(f'Training {class_} set length: {training_length}')
        print(
            f'Validation {class_} set length: {len(development_class_image_paths) - training_length}')

        training_class_image_paths = development_class_image_paths[:training_length]
        validation_class_image_paths = development_class_image_paths[training_length:]
        print(
            f'Training {class_} image count: {len(training_class_image_paths)}')
        print(
            f'Validation {class_} image count: {len(validation_class_image_paths)}')

        split_folder_name_split_image_class_paths_dict = {
            TRAIN_FOLDER_NAME: training_class_image_paths,
            VALIDATION_FOLDER_NAME: validation_class_image_paths,
            TEST_FOLDER_NAME: test_class_image_paths
        }

        print()

        for split_folder_name, split_class_image_paths in split_folder_name_split_image_class_paths_dict.items():
            split_path = split_data_path / split_folder_name
            destination_path: Path = split_path / class_

            '''
            Only append destination paths and make the split folders on the
            first class_ iteration. We don't want duplicate folders.
            '''
            if class_index == 0:
                destination_paths.append(str(split_path))
                split_path.mkdir(parents=False, exist_ok=True)

            '''
            Make the class folder in each split folder
            '''
            destination_path.mkdir(parents=False, exist_ok=True)

            if allow_move_or_copy:
                print(
                    f'{copy_move_str} {split_folder_name} files from {dataset_path} to {destination_path}')
                for split_class_image_path in split_class_image_paths:
                    if move == True:
                        shutil.move(split_class_image_path,
                                    str(destination_path))
                    else:
                        shutil.copy(split_class_image_path,
                                    str(destination_path))

        '''
        If not the last iteration and allow_move_or_copy...
        '''
        if class_index != len(class_list) and allow_move_or_copy:
            print()

    return destination_paths


In [54]:
training_dataset_path, validation_dataset_path, test_dataset_path = stratified_train_val_test_split_into_folders()

display([training_dataset_path, validation_dataset_path, test_dataset_path])

Not copying files as C:\Users\Michael Metz\Documents\Education\MEng Software\2023-01 - 2023-04 Winter\ENEL 645\BreakHis\BreaKHis_v1\BreaKHis_v1\histology_slides\breast\split_data already esists
Development benign set length: 1736
Test benign set length: 744
Development benign image count: 1736
Test benign image count: 744
Training benign set length: 1488
Validation benign set length: 248
Training benign image count: 1488
Validation benign image count: 248


Development malignant set length: 3800
Test malignant set length: 1629
Development malignant image count: 3800
Test malignant image count: 1629
Training malignant set length: 3257
Validation malignant set length: 543
Training malignant image count: 3257
Validation malignant image count: 543




['C:\\Users\\Michael Metz\\Documents\\Education\\MEng Software\\2023-01 - 2023-04 Winter\\ENEL 645\\BreakHis\\BreaKHis_v1\\BreaKHis_v1\\histology_slides\\breast\\split_data\\training',
 'C:\\Users\\Michael Metz\\Documents\\Education\\MEng Software\\2023-01 - 2023-04 Winter\\ENEL 645\\BreakHis\\BreaKHis_v1\\BreaKHis_v1\\histology_slides\\breast\\split_data\\validation',
 'C:\\Users\\Michael Metz\\Documents\\Education\\MEng Software\\2023-01 - 2023-04 Winter\\ENEL 645\\BreakHis\\BreaKHis_v1\\BreaKHis_v1\\histology_slides\\breast\\split_data\\test']